In [19]:
import pandas as pd 

df=pd.read_csv("small_dataset.csv")
df.dropna(inplace=True)
df["Context"] = df["ProductName"] +" :\n\n" +df["Description"]

df.head()

from langchain.schema import Document

def df_to_langchain_documents(df):
    documents = []
    
    for index, row in df.iterrows():
        page_content = row["Context"]
        
        metadata = row.drop(labels=["Context","Description"]).to_dict()
        
        documents.append(Document(page_content=page_content, metadata=metadata))
    
    return documents


In [20]:
elastic_content=df_to_langchain_documents(df)

In [21]:
elastic_content[0].page_content

'DKNY Unisex Black & Grey Printed Medium Trolley Bag :\n\nBlack and grey printed medium trolley bag, secured with a TSA lockOne handle on the top and one on the side, has a trolley with a retractable handle on the top and four corner mounted inline skate wheelsOne main zip compartment, zip lining, two compression straps with click clasps, one zip compartment on the flap with three zip pocketsWarranty: 5 yearsWarranty provided by Brand Owner / Manufacturer'

In [22]:
elastic_content[0].metadata

{'ProductID': 10017413,
 'ProductName': 'DKNY Unisex Black & Grey Printed Medium Trolley Bag',
 'ProductBrand': 'DKNY',
 'Gender': 'Unisex',
 'Price (INR)': 11745,
 'NumImages': 7,
 'PrimaryColor': ' Black'}

In [23]:
import os
from elasticsearch import Elasticsearch
client=Elasticsearch(hosts=os.environ["ELASTIC_CLOUD_ID"],api_key=os.environ["ELASTIC_API_KEY"], request_timeout=300)

In [24]:
from langchain_core.documents import Document
from langchain_elasticsearch import ElasticsearchStore
from langchain_openai import OpenAIEmbeddings


embeddings = OpenAIEmbeddings()
# vectorstore = ElasticsearchStore.from_documents(
#     elastic_content,
#     embeddings,
#     index_name="myntra-self-retrevier-index",
#     es_connection=client
# )

In [25]:
vectorstore=ElasticsearchStore(index_name="myntra-self-retrevier-index",embedding=embeddings,es_connection=client)

In [26]:
vectorstore

In [36]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI

metadata_field_info = [
    AttributeInfo(
        name="ProductName",
        description="The name of the product",
        type="string",
    ),
    AttributeInfo(
        name="ProductBrand",
        description="The brand or manufacturer of the product",
        type="string",
    ),
    AttributeInfo(
        name="Gender",
        description="The gender the product is intended for, if specified",
        type="string",
    ),
    AttributeInfo(
        name="Price (INR)",
        description="The price of the product in Indian Rupees (INR)",
        type="integer",
    ),
    AttributeInfo(
        name="NumImages",
        description="The number of images associated with the product",
        type="integer",
    ),
    AttributeInfo(
        name="PrimaryColor",
        description="The primary color of the product",
        type="string",
    ),
]

document_content_description = "Brief description of the Myntra Products"
llm = ChatOpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm, vectorstore, document_content_description, metadata_field_info, verbose=True
)

In [37]:
retriever.invoke("Show me all DKNY products available in black, priced above ₹10,000.")

[]

In [38]:
retriever.invoke("Find EthnoVogue women's products that are beige and cost less than ₹6,000.")

[]

In [39]:
retriever.invoke("Can you list SPYKAR women's clothing in pink with more than 5 images?")

[]

In [40]:
retriever.invoke("I'm looking for a Raymond blue men's suit that costs under ₹6,000 and has at least 5 images.")

[]

In [41]:
retriever.invoke("Do you have any DKNY unisex trolley bags in black or grey priced between ₹11,000 and ₹12,000?")

OutputParserException: Parsing text
```json
{
    "query": "DKNY unisex trolley bags",
    "filter": "and(eq(\"ProductBrand\", \"DKNY\"), eq(\"Gender\", \"unisex\"), eq(\"PrimaryColor\", \"black\"), or(eq(\"PrimaryColor\", \"grey\"), and(gte(\"Price (INR)\", 11000), lte(\"Price (INR)\", 12000)))"
}
```
 raised following error:
Unexpected token Token('$END', '') at line 1, column 174.
Expected one of: 
	* COMMA
	* RPAR


In [33]:
retriever.invoke("Find EthnoVogue kurta sets for women in grey with a price lower than ₹6,000.")

[]

In [34]:
retriever.invoke("I'm searching for Parx men's casual shirts in brown or white with less than 6 images.")

[]

In [35]:
retriever.invoke("Are there any SPYKAR products for women in pink that are priced under ₹1,000?")

[]